In [1]:
#### relative data, where the replicates were considered, their OD relative ratio taken and the mean evaluated
#### modification : remove wells that correspond to high variability and bistability (three such wells)

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input

from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from matplotlib import pyplot
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")


import seaborn as sns

table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/A8.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/iHiV7.pkl")
iHiV = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/iHiV8.pkl")
VHred = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/VH7.pkl")
tableg = table.T

#iFT = pd.read_pickle("iFT4.pkl")

VH = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/VH8.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/B8.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(8):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(8):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


#### get all data -- design matrix, steady state abundances, and list of wells present

#sst = pd.read_pickle("Friedman-data-sst-rel.pkl")
#sst = pd.read_pickle("will-ludd-log10-properdata.pkl")
#sstv3 = sst.copy()

laml = [1e-20,1e-10,1e-5,.01,.02,.03,.04,.05,.06,.0650,.0675,.07,0.08,0.1,0.13,0.15,0.18,0.2,0.5,1.0,2.0,3.0,5.0,10.0,20.0,50.0,100.0]
from sklearn.metrics import r2_score

g = np.array([[1, 0], [-1, 1]])

for i in range(6):
    g = np.block([[g, np.zeros(g.shape)], [-g, g]])

G = g
X = np.linalg.inv(G)


In [3]:
print(iHiVred.shape, X.shape)
Xred = X

(128, 128) (128, 128)


In [4]:
list_present  = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/list-present-friedman.pkl")
sst_rel = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/Friedman-v3/Friedman-data-sst-relv2.pkl")

sp_list_provided = ["Ea","Pa","Pch","Pci","Pf","Pp","Pv","Sm"] 
sst_rel2 = sst_rel.rename(columns={"Sm":0,"Pv":1,"Pp":2,"Pf":3,"Pci":4,"Pch":5,"Pa":6,"Ea":7})
sstv3 = sst_rel2[[0,1,2,3,4,5,6,7,"well index"]]

sstv4 = sstv3.fillna(0)

sstv4.head(3)

,0,1,2,3,4,5,6,7,well index
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2


In [5]:
n = 2**7
cv_splitsl = [3,5]

In [6]:
to_be_removed = {73,164,193}
list_presentv2 = [item for item in list_present if item not in to_be_removed ]

In [7]:
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

In [8]:
dataTR = pd.read_pickle("traindataRELfried.pkl")
dataTE = pd.read_pickle("testdataRELfried.pkl")

In [9]:
err = []
errlin = []
betaspred  = [] 
for cv_splits in cv_splitsl: 
    datatrain = dataTR[dataTR["k-fold"]==cv_splits]
    datatest = dataTE[dataTE["k-fold"]==cv_splits]
    for speciesc in range(8):
        datatousetrain = datatrain[datatrain["species"]==speciesc]
        datatousetest = datatest[datatest["species"]==speciesc]
        abunvec = pd.DataFrame(sstv4[speciesc].values).T[species_ones[speciesc]].T
        abunvec.index = np.arange(0,2**7,1)
        for lmda in laml:
            for iterii in range(cv_splits):
                rinew = datatousetrain[datatousetrain["fold number"]==iterii]["train data"].values[0]
                rileft = datatousetest[datatousetest["fold number"]==iterii]["test data"].values[0]
                y3 = abunvec.T[rinew].T[0].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = iHiVred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(iHiVred,x.ravel())
                err.append([lmda,speciesc,yrecon[rileft],rileft,rinew,cv_splits])
                betaspred.append([lmda,cv_splits,speciesc,x.ravel()])
                y3 = abunvec.T[rinew].T[0].values                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
                D = Xred[rinew,:]
                s2 = np.asarray([y3]).astype(float)
                s3 = s2.T
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(Xred,x.ravel())
                errlin.append([lmda,speciesc,yrecon[rileft],rileft,rinew,cv_splits])

In [10]:
errdf= pd.DataFrame(err)
errlindf = pd.DataFrame(errlin)

In [11]:
pd.to_pickle(errdf,"CS-rel-WH.pkl")
pd.to_pickle(errlindf,"CS-rel-sigma.pkl")